In [29]:
# Dependencies and Setup
import requests
import time
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
import sklearn.datasets as dta
import scipy.stats as st
from scipy.stats import sem
from scipy.stats import linregress
import seaborn as sns

In [30]:
Vaccine_dose1_df = pd.read_csv('C:/Users/panka/OneDrive/Desktop/Class Folder/Github/project_3_for_COVID19/Datasets_ETLwork/Vaccine_record_Government_of_Canada/vaccination-coverage-byAgeAndSex-overTimeDownload.csv',delimiter=',', header=0, na_values='NaN')
Vaccine_dose1_df['Date'] = pd.to_datetime(Vaccine_dose1_df['Date'])
Vaccine_dose1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81228 entries, 0 to 81227
Data columns (total 5 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   Province                                     81228 non-null  object        
 1   Date                                         81228 non-null  datetime64[ns]
 2   Sex                                          81228 non-null  object        
 3   Age                                          81228 non-null  object        
 4   Cummulative number of people (Vaccinedose1)  81228 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.1+ MB


In [31]:
# Remove NaN values from specific columns
# remove NaN values and other common missing value representations from the 'column_name' column
missing_values = ['na', 'N/A', 'null', '']

# Replace missing value representations with NaN
Vaccine_dose1_df['Cummulative number of people (Vaccinedose1)'] = Vaccine_dose1_df['Cummulative number of people (Vaccinedose1)'].replace(missing_values, float('nan'))

# Drop NaN values from the column
Vaccine_dose1_df.dropna(subset=['Cummulative number of people (Vaccinedose1)'], inplace=True)

# convert the 'Cummulative number of people (Vaccinedose1)' column to int64
Vaccine_dose1_df['Cummulative number of people (Vaccinedose1)'] = Vaccine_dose1_df['Cummulative number of people (Vaccinedose1)'].astype('int64')

In [32]:
# Specify the entry you want to remove
# entry_to_remove = 'Canada'

# Remove all rows with the specified entry in the 'Name' column
# Vaccine_dose1_df = Vaccine_dose1_df[Vaccine_dose1_df['Province'] != entry_to_remove]
Vaccine_dose1_df = Vaccine_dose1_df.rename(columns={'Cummulative number of people (Vaccinedose1)': 'Cumulative number of people (Vaccinedose1)'})

# Display the modified dataframe
Vaccine_dose1_df

,Province,Date,Sex,Age,Cumulative number of people (Vaccinedose1)
0,Canada,2020-12-19,All sexes,0–15,2
1,Canada,2020-12-19,All sexes,16–69,11766
2,Canada,2020-12-19,All sexes,70–74,174
3,Canada,2020-12-19,All sexes,75–79,85
4,Canada,2020-12-19,All sexes,80+,335
...,...,...,...,...,...
81223,Nunavut,2023-04-23,Not reported,70–79,0
81224,Nunavut,2023-04-23,Not reported,80+,0
81225,Nunavut,2023-04-23,Not reported,All ages,0
81226,Nunavut,2023-04-23,Not reported,Not reported,0


In [33]:
# # Create a boolean mask to filter rows based on 'Sex' column
# mask = (Vaccine_dose1_df['Sex'] == 'm') | (Vaccine_dose1_df['Sex'] == 'f') | (Vaccine_dose1_df['Sex'] == 'Other')
# # Apply the mask to the DataFrame
# filtered_df = Vaccine_dose1_df[mask]
age_mask = (Vaccine_dose1_df['Age'] == '0–4') | (Vaccine_dose1_df['Age'] == '5–11') | (Vaccine_dose1_df['Age'] == '12–17') | (Vaccine_dose1_df['Age'] == '18–29') | (Vaccine_dose1_df['Age'] == '30–39') | (Vaccine_dose1_df['Age'] == '40–49') | (Vaccine_dose1_df['Age'] == '50–59') | (Vaccine_dose1_df['Age'] == '60–69') | (Vaccine_dose1_df['Age'] == '70–79') | (Vaccine_dose1_df['Age'] == '80+')
filtered_rows_df = Vaccine_dose1_df[age_mask]
filtered_rows_df

,Province,Date,Sex,Age,Cumulative number of people (Vaccinedose1)
4,Canada,2020-12-19,All sexes,80+,335
12,Canada,2020-12-19,f,80+,241
20,Canada,2020-12-19,m,80+,94
28,Canada,2020-12-19,Unknown,80+,0
36,Canada,2020-12-19,Not reported,80+,0
...,...,...,...,...,...
81220,Nunavut,2023-04-23,Not reported,40–49,0
81221,Nunavut,2023-04-23,Not reported,50–59,0
81222,Nunavut,2023-04-23,Not reported,60–69,0
81223,Nunavut,2023-04-23,Not reported,70–79,0


In [34]:
cumulative_work = filtered_rows_df.groupby('Age').max()[['Cumulative number of people (Vaccinedose1)', 'Date']].sort_values(['Cumulative number of people (Vaccinedose1)'], ascending = False).reset_index()
cumulative_work_df = cumulative_work[['Age', 'Cumulative number of people (Vaccinedose1)']]
cumulative_work_df = cumulative_work_df.set_index('Age')
cumulative_work_df.head(13)

,Cumulative number of people (Vaccinedose1)
Age,
18–29,5336635
30–39,4812249
60–69,4679616
50–59,4670283
40–49,4471985
70–79,3190841
12–17,2176233
80+,1804052
0–4,179936


In [36]:
filtered_rows_df.to_csv("Covid19_dataset2UPDATED(Vaccination1)_project3.csv", index=False)